In [14]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [37]:
import numpy 
import tensorflow 
import tflearn
import random
import json
import pickle 

In [38]:
with open("intents.json") as file:
    data = json.load(file)
print(data["intents"]) # Reading the file 


[{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am leaving', 'Have a Good day'], 'responses': ['Sad to see you fo :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'how old is tim', 'what is your age', 'how old are you', 'age?'], 'responses': ['I am 18 years old!', '18 years young!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Tim', "I'm Tim!", "I'm Tim aka Tech with Tim"], 'context_set': ''}, {'tag': 'shop', 'patterns': ['Id like to buy something', 'whats on the menu', 'what do you recommend?', 'could I get something'], 'responses': ['We sell chocolate chip cookies for $2!', 'Cookies are on the menu!'], 'con

In [39]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(pattern)
        docs_y.append(intent["tag"])
        
    if intent["tag"] not in labels:
        labels.append(intent["tag"])
        


In [21]:
print(labels)
print(words)
print(docs_x)
print(docs_y)

['greeting', 'goodbye', 'age', 'name', 'shop', 'hours', 'buy']
['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'whats', 'up', 'cya', 'See', 'you', 'later', 'Goodbye', 'I', 'am', 'leaving', 'Have', 'a', 'Good', 'day', 'how', 'old', 'how', 'old', 'is', 'tim', 'what', 'is', 'your', 'age', 'how', 'old', 'are', 'you', 'age', '?', 'what', 'is', 'your', 'name', 'what', 'should', 'I', 'call', 'you', 'whats', 'your', 'name', '?', 'Id', 'like', 'to', 'buy', 'something', 'whats', 'on', 'the', 'menu', 'what', 'do', 'you', 'recommend', '?', 'could', 'I', 'get', 'something', 'when', 'are', 'you', 'guys', 'open', 'what', 'are', 'your', 'hours', 'hours', 'of', 'operation', 'can', 'I', 'buy', 'something', 'how', 'much', 'is', 'a', 'cookie']
['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'whats up', 'cya', 'See you later', 'Goodbye', 'I am leaving', 'Have a Good day', 'how old', 'how old is tim', 'what is your age', 'how old are you', 'age?', 'what is yo

In [40]:
words = [stemmer.stem(w.lower()) for w in words]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w) for w in doc if w !="?"]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
            
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)

with open("data.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)


In [41]:
print(training.shape)
print(output.shape)

(28, 49)
(28, 7)


In [42]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


In [43]:
model.fit(training,output, n_epoch=100, batch_size = 8, show_metric=True)
model.save("model.tflearn")

Training Step: 399  | total loss: 1.87759 | time: 0.010s
| Adam | epoch: 100 | loss: 1.87759 - acc: 0.2135 -- iter: 24/28
Training Step: 400  | total loss: 1.86566 | time: 0.013s
| Adam | epoch: 100 | loss: 1.86566 - acc: 0.2422 -- iter: 28/28
--
INFO:tensorflow:/Users/abhishekde/Jupyter_projects/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [45]:
with open("data.pickle", "rb") as rf:
    words, labels, training, output = pickle.load(rf)
    
try:
    model.load("model.tflearn")
except:
    model.fit(training,output, n_epoch=100, batch_size = 8, show_metric=True)
    model.save("model.tflearn")
    

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /Users/abhishekde/Jupyter_projects/model.tflearn


In [50]:
def bag_of_words(s,words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
                
                
    return numpy.array(bag)

In [51]:
def chat():
    print("Start talking with the bot! (type 'quit' to stop)")
    while True:
        inp = input("You: ")
        if inp.lower()=="quit":
            break
            
        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        
        if results[results_index] > 0.7:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
        else:
            print("I didn't get that. Try again!")

                
        print(random.choice(responses))

In [52]:
chat()

Start talking with the bot! (type 'quit' to stop)
You: Hi
Hello!
You: I am Tim
Good to see you again!
You: quit


In [55]:
out_empty = [0 for _ in range(len(labels))]
print(out_empty)

[0, 0, 0, 0, 0, 0, 0]
